In [23]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-36mlpiio/gym-walk_61d3c025de2b4a6fa5fcfbb9dbd549ed
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-36mlpiio/gym-walk_61d3c025de2b4a6fa5fcfbb9dbd549ed
  Resolved https://github.com/mimoralea/gym-walk to commit 5999016267d6de2f5a63307fb00dfd63de319ac1
  Preparing metadata (setup.py) ... done


In [24]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [25]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [26]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [57]:
# ipython-input-54-33994a74429d

def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            # Access the action from the policy dictionary
            action = pi(state)
            state, _, done, h = env.step(action)
            steps += 1
        results.append(state == goal_state)
    return np.mean(results)

In [37]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
#goal_state = 6
#LEFT, RIGHT = range(2)

In [38]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [39]:
import numpy as np
def decay_schedule(init_value, min_value,decay_ratio, max_steps, log_start=-2, log_base=10):
  decay_steps = int(max_steps * decay_ratio)
  rem_steps = max_steps - decay_steps
  values = np.logspace (log_start, 0, decay_steps,base=log_base, endpoint=True) [::-1]
  values =(values - values.min()) / (values.max()-values.min())
  values = (init_value - min_value) * values + min_value
  values = np.pad(values, (0, rem_steps), 'edge')
  return values

In [40]:
from itertools import count # Import the count function from itertools

def generate_trajectory(select_action, Q, epsilon,env, max_steps=200): # Corrected order of arguments
  done, trajectory = False, []
  while not done:
    state = env.reset()
    for t in count(): # Use the imported count function
      action = select_action(state, Q, epsilon)
      next_state, reward, done, _ = env.step(action)
      experience = (state, action, reward, next_state, done)
      trajectory.append(experience)
      if done:
        break
      if t >= max_steps - 1:
        trajectory = []
        break
      state = next_state
  return np.array(trajectory, dtype=object)

In [58]:
# ipython-input-42-3cc6b3253f36

import numpy as np
from tqdm import tqdm

def mc_control(env, gamma=1.0,
                init_alpha=0.5, min_alpha=0.01, alpha_decay_ratio=0.5,
                init_epsilon=1.0, min_epsilon=0.1, epsilon_decay_ratio=0.9,
                n_episodes=3000, max_steps=200, first_visit=True):
  nS, nA = env.observation_space.n, env.action_space.n

  # Write your code here
  discounts = np.logspace(0, max_steps, num=max_steps, base=gamma, endpoint=False)
  alphas = decay_schedule(init_alpha, min_alpha, alpha_decay_ratio, n_episodes)
  epsilons = decay_schedule(init_epsilon, min_epsilon, epsilon_decay_ratio, n_episodes)
  pi_track = []
  Q = np.zeros((nS, nA), dtype=np.float64)
  Q_track = np.zeros((n_episodes, nS, nA), dtype=np.float64)
  select_action = lambda state, Q, epsilon: np.argmax(Q[state]) if np.random.random() > epsilon else np.random.randint(len(Q[state]))

  for e in tqdm(range(n_episodes), leave=False):
    trajectory = generate_trajectory(select_action, Q, epsilons[e], env, max_steps)
    visited = np.zeros((nS, nA), dtype=bool)
    for t, (state, action, reward, _, _) in enumerate(trajectory):
      if visited[state][action] and first_visit:
        continue
      visited[state][action] = True
      n_steps = len(trajectory[t:])
      G = np.sum(discounts[:n_steps] * trajectory[t:, 2])
      Q[state][action] = Q[state][action] + alphas[e] * (G - Q[state][action])
    Q_track[e] = Q
    pi_track.append(np.argmax(Q, axis=1))
  V = np.max(Q, axis=1)
  # Return the policy as a function instead of a dictionary
  pi = lambda s: {s: a for s, a in enumerate(np.argmax(Q, axis=1))}[s]

  # return Q, V, pi, Q_track, pi_track
  return Q, V, pi

In [42]:
import numpy as np
from tqdm import tqdm

def mc_control (env, gamma = 1.0,
                init_alpha = 0.5,min_alpha = 0.01, alpha_decay_ratio = 0.5,
                init_epsilon = 1.0, min_epsilon = 0.1, epsilon_decay_ratio = 0.9,
                n_episodes = 3000, max_steps = 200, first_visit = True):
  nS, nA = env.observation_space.n, env.action_space.n

  #Write your code here
  discounts=np.logspace(0,max_steps,num=max_steps, base=gamma, endpoint=False)
  alphas = decay_schedule(init_alpha, min_alpha, alpha_decay_ratio,n_episodes)
  epsilons=decay_schedule(init_epsilon, min_epsilon, epsilon_decay_ratio,n_episodes)
  pi_track=[]
  Q = np.zeros((nS, nA),dtype=np.float64)
  Q_track = np.zeros((n_episodes,nS,nA),dtype=np.float64 )
  select_action = lambda state, Q, epsilon : np.argmax(Q[state]) if np.random.random()> epsilon else np.random.randint(len(Q[state]))

  for e in tqdm(range(n_episodes),leave=False):
    trajectory = generate_trajectory(select_action,Q, epsilons[e],env, max_steps)
    visited = np.zeros((nS, nA), dtype=bool)
    for t, (state, action, reward,_,_) in enumerate(trajectory):
      if visited[state][action] and first_visit:
        continue
      visited[state][action]=True
      n_steps=len(trajectory[t:])
      G=np.sum(discounts[:n_steps] * trajectory[t:,2])
      Q[state][action] = Q[state][action] + alphas[e] * (G-Q[state][action])
    Q_track[e]=Q
    pi_track.append(np.argmax(Q,axis=1))
  V=np.max(Q, axis=1)
  # Change here: Return the policy as a dictionary instead of a lambda function
  pi = {s: a for s, a in enumerate(np.argmax(Q, axis=1))}

  # return Q, V, pi, Q_track, pi_track
  return Q, V, pi

In [43]:
optimal_Q, optimal_V, optimal_pi = mc_control (env,n_episodes = 3000)
print('\nName: Javith Farkhan S     Register Number: 212221240017')
print_state_value_function(optimal_Q, P, n_cols=4, prec=2, title='Action-value function:')
print_state_value_function(optimal_V, P, n_cols=4, prec=2, title='State-value function:')
print_policy(optimal_pi, P)


Name: Javith Farkhan S     Register Number: 212221240017
Action-value function:
| 00 [0.38 0.28 0.3  0.24] | 01 [0.12 0.07 0.09 0.26] | 02 [0.21 0.06 0.06 0.06] | 03 [0.04 0.   0.   0.  ] |
| 04 [0.4  0.19 0.19 0.15] |           | 06 [0.06 0.07 0.18 0.01] |           |
| 08 [0.17 0.16 0.24 0.43] | 09 [0.2  0.48 0.18 0.15] | 10 [0.21 0.29 0.36 0.14] |           |
|           | 13 [0.25 0.32 0.6  0.31] | 14 [0.42 0.81 0.49 0.5 ] |           |
State-value function:
| 00   0.38 | 01   0.26 | 02   0.21 | 03   0.04 |
| 04    0.4 |           | 06   0.18 |           |
| 08   0.43 | 09   0.48 | 10   0.36 |           |
|           | 13    0.6 | 14   0.81 |           |
Policy:
| 00      < | 01      ^ | 02      < | 03      < |
| 04      < |           | 06      > |           |
| 08      ^ | 09      v | 10      > |           |
|           | 13      > | 14      v |           |
